# Using neural classification
As has been proven by [Wang (2017)](https://arxiv.org/abs/1705.00648), neural classifiers carry better results than non-neural classifiers when detecting fake news. However, it is unknown how well neural networks classify fake news when using previously mentioned text embeddings. 
In this notebook, the second research question will be answered: *how well do neural network architecture classify fake news compared to non-neural classification algorithms?*

<hr>

In [1]:
# General imports
import pandas as pd
import numpy as np
import json
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Set offline mode for plotly
init_notebook_mode(connected = True)

In [2]:
def generate_trace(accuracies, embedding, color):
    accuracies = np.array(accuracies)
    y = [np.average(row) for row in accuracies]
    y_upper = [row.max() for row in accuracies]
    y_lower = [row.min() for row in accuracies]
    y_lower = y_lower[::-1]

    return [go.Scatter(
            x = x + x_rev,
            y = y_upper + y_lower,
            fill = 'tozerox',
            fillcolor = 'rgba(' + color + ', 0.2)',
            line = dict(color = 'rgba(' + color + ', 0)'),
            showlegend = False,
            name = embedding,
        ),
        go.Scatter(
            x = x,
            y = y,
            line = dict(color='rgb(' + color + ')'),
            mode = 'lines+markers',
            name = embedding,
        )
    ]

In [5]:
# Baseline and highest performance
x = list(range(2, 37))
baselines = [
    go.Scatter(
        x = x,
        y = [0.4428 for i in x],
        line = dict(color='rgb(0,0,0)'),
        mode = 'lines',
        name = 'Majority vote',
    ), 
    go.Scatter(
        x = x,
        y = [0.4903 for i in x],
        mode = 'lines',
        name = 'Khurana (2017)',
    )
]

In [6]:
with open('results.json') as json_data:
    results = json.load(json_data)
    
    x = list(range(2, 37))
    x_rev = x[::-1]
    colors = ['231,107,243', '81, 45, 168', '205,12,24', '0,176,246', '0,100,80']
    
    traces = list(
        np.array([
            generate_trace(
                list(results[list(results.keys())[i]]['3']['bilstm'].values()), list(results.keys())[i], colors[i]) 
            for i in range(len(results.keys()))
        ]).flatten()
    )
    
    data = traces + baselines

    layout = go.Layout(
        title = 'Bi-LSTM test set accuracy of padded datasets with variable maximum lengths',
        paper_bgcolor = 'rgb(255,255,255)',
        plot_bgcolor = 'rgb(229,229,229)',
        xaxis = dict(
            gridcolor = 'rgb(255,255,255)',
            range = [2,36],
            showgrid = True,
            showline = False,
            showticklabels = True,
            tickcolor = 'rgb(127,127,127)',
            ticks = 'outside',
            zeroline = False
        ),
        yaxis=dict(
            range = [0.40, 0.54],
            gridcolor='rgb(255,255,255)',
            showgrid = True,
            showline = False,
            showticklabels = True,
            tickcolor = 'rgb(127,127,127)',
            ticks = 'outside',
            zeroline = False
        ),
    )

    fig = go.Figure(data = data, layout = layout)

    iplot(fig)

In [7]:
with open('results.json') as json_data:
    results = json.load(json_data)
    
    x = list(range(2, 37))
    x_rev = x[::-1]
    colors = ['231,107,243', '81, 45, 168', '205,12,24', '0,176,246', '0,100,80']
    
    traces = list(
        np.array([
            generate_trace(
                list(results[list(results.keys())[i]]['3']['cnn'].values()), list(results.keys())[i], colors[i]) 
            for i in range(len(results.keys()))
        ]).flatten()
    )
    
    data = traces + baselines

    layout = go.Layout(
        title = 'CNN test set accuracy of padded datasets with variable maximum lengths',
        paper_bgcolor = 'rgb(255,255,255)',
        plot_bgcolor = 'rgb(229,229,229)',
        xaxis = dict(
            gridcolor = 'rgb(255,255,255)',
            range = [2,36],
            showgrid = True,
            showline = False,
            showticklabels = True,
            tickcolor = 'rgb(127,127,127)',
            ticks = 'outside',
            zeroline = False
        ),
        yaxis=dict(
            range = [0.40, 0.54],
            gridcolor='rgb(255,255,255)',
            showgrid = True,
            showline = False,
            showticklabels = True,
            tickcolor = 'rgb(127,127,127)',
            ticks = 'outside',
            zeroline = False
        ),
    )

    fig = go.Figure(data = data, layout = layout)

    iplot(fig)

<hr>

### References

```
@article{DBLP:journals/corr/Wang17j,
  author    = {William Yang Wang},
  title     = {"Liar, Liar Pants on Fire": {A} New Benchmark Dataset for Fake News
               Detection},
  journal   = {CoRR},
  volume    = {abs/1705.00648},
  year      = {2017},
  url       = {http://arxiv.org/abs/1705.00648},
  archivePrefix = {arXiv},
  eprint    = {1705.00648},
  timestamp = {Mon, 13 Aug 2018 16:48:58 +0200},
  biburl    = {https://dblp.org/rec/bib/journals/corr/Wang17j},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}
```